<a href="https://colab.research.google.com/github/GourabNath/Research-Paper-Search-Algorithm-to-indentify-similar-reviews/blob/master/Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import glob
import os
os.getcwd()

'/content'

In [57]:
from google.colab import files
data1 = files.upload()

Saving table1_final.csv to table1_final (1).csv


In [0]:
import io
df2 = pd.read_csv(io.BytesIO(data1['table1_final.csv']))
# Dataset is now stored in a Pandas Dataframe

In [0]:
table1 = df2

In [5]:
table1.head(2)

,Review_text,phone,amazon,android,backup,battery,beast,bite,brand,budget,call,camera,charge,charger,dash,deal,delivery,design,device,display,experience,feature,flagship,game,headphone,heat,issue,lag,life,look,market,mobile,money,performance,phone.1,photo,picture,price,problem,processor,product,purchase,quality,ram,range,review,sale,screen,service,smartphone,software,sound,speaker,time,update,usage,user,value,video
0,Best phone ever,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,One of the best phone!,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
import nltk

#De-Emojify
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

#Remove Special Symbols other than (.), (,) and (!)
def removeSymbols(inputString):
    return(inputString.translate(str.maketrans("", "", "#$%&\'()*+-/:;<=>?@[\\]^_`{|}~")))

#Remove (.), (,) and (!) for later stage
def remove_dot_comma_excl(inputString):
    s = inputString.translate(str.maketrans(".", " ", ""))
    s = s.translate(str.maketrans(",", " ", ""))
    s = s.translate(str.maketrans("!", " ", ""))
    return(' '.join(s.split()))


#Text cleaning function -> Return a list of tokenized sentence
#(Which will be used for word tokenizarion followed by POS tagging)

def clean_review1(review):
    r = review.lower()
    r = removeSymbols(r)
    r = deEmojify(r)
    r = nltk.sent_tokenize(r)

    tokens = []
    for sentence in r:
        #print(sentence)
        if '.' in sentence:
            sentence = sentence.split('.')
            for _s in sentence:
                if _s == '':
                    continue
                else:
                    tokens.append(_s.strip(' .,!')) 
        else:
            if len(sentence) > 0:
                    tokens.append(sentence)
    return(tokens)

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
# clean_review1('best phone ever ....heelo.')
clean_review1("one plus 6 is extremely worth for it's price and moreover in Amazon sale it's even beyond that. 35k for this phone is worth buying but in Amazon sale it costs me 27k. it's just mind blowing deal for me.\n.\nA fast device with perfect photo clicking quality. Definitely the best phone to buy")

['one plus 6 is extremely worth for its price and moreover in amazon sale its even beyond that',
 '35k for this phone is worth buying but in amazon sale it costs me 27k',
 'its just mind blowing deal for me',
 'a fast device with perfect photo clicking quality',
 'definitely the best phone to buy']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


pandas.core.frame.DataFrame

In [12]:
table1.Review_text[2]
# clean_review1(a)

"one plus 6 is extremely worth for it's price and moreover in Amazon sale it's even beyond that. 35k for this phone is worth buying but in Amazon sale it costs me 27k. it's just mind blowing deal for me.\r\n.\r\nA fast device with perfect photo clicking quality. Definitely the best phone to buy"

In [0]:
def check_nn_adj(sent,noun):
    #sent = clean_review1(sent)
#     print(sent)
    adj=[];dist=100;
    if noun in sent:
#         print(noun, sent)
        postag = nltk.pos_tag(nltk.word_tokenize(sent));
#         print(postag)
        for _i in range(0,len(postag)):
            if postag[_i][1] in ['JJ','JJS']:
                adj.append([postag[_i][0],_i])
    return (adj)

In [14]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [15]:
check_nn_adj('Definitely the best phone to buy', 'phone')
# nltk.pos_tag(nltk.word_tokenize('not a good phone'))

[('not', 'RB'), ('a', 'DT'), ('good', 'JJ'), ('phone', 'NN')]

In [0]:
def check_nn_adv(sent,noun):
    #sent = clean_review1(sent)
#     print(sent)
    adv=[];dist=100;
    if noun in sent:
#         print(noun, sent)
        postag = nltk.pos_tag(nltk.word_tokenize(sent));
#         print(postag)
        for _i in range(0,len(postag)):
            if postag[_i][1] in ['RB','RBR','RBS']:
                adv.append([postag[_i][0],_i])
    return (adv)

In [17]:
check_nn_adv('a fast device with perfect photo clicking quality', 'phone')

[]

In [0]:
def check_nn_vrb(sent,noun):
    #sent = clean_review1(sent)
#     print(sent)
    vrb=[];dist=100;
    if noun in sent:
#         print(noun, sent)
        postag = nltk.pos_tag(nltk.word_tokenize(sent));
#         print(postag)
        for _i in range(0,len(postag)):
            if postag[_i][1] in ['VBD','VBG','VPN']:
                vrb.append([postag[_i][0],_i])
    return (vrb)

In [19]:
check_nn_vrb('a fast device with perfect photo clicking quality', 'device')

[['clicking', 6]]

In [0]:
# Nearest Verb or adjective

def nearest(postag,noun,adj,vrb):
    adj_ind=[]; nn_ind=[]; reslist=[]
    for i,pos in enumerate(postag):
        if postag[i][0].lower() == noun:
            nn_ind.append(i+1)
#     print('n ==  ',nn_ind)
    least_dist=100
    pos=''
    least_wrd=''
    ptag=''
    for i in nn_ind:
        flg=1
        for j in adj:
#             print(i, j[1])
            dist = abs(i - j[1])
            if least_dist > dist:
                least_dist = dist
                least_wrd = j[0]
                pos = j[1]
                ptag = 'ADJ'
        for k in vrb:
            dist = abs(i - k[1])
            if least_dist > dist:
                least_dist = dist
                least_wrd = k[0]
                pos = k[1]
                ptag = 'VRB'
#     print([least_dist,least_wrd])
        if flg != '':
            reslist=[least_wrd,pos,ptag]
        else:
            reslist = []            
            
    return reslist

In [23]:
dfadj = table1
dfvrb = table1
dfadj.head(2)

,Review_text,phone,amazon,android,backup,battery,beast,bite,brand,budget,call,camera,charge,charger,dash,deal,delivery,design,device,display,experience,feature,flagship,game,headphone,heat,issue,lag,life,look,market,mobile,money,performance,phone.1,photo,picture,price,problem,processor,product,purchase,quality,ram,range,review,sale,screen,service,smartphone,software,sound,speaker,time,update,usage,user,value,video
0,Best phone ever,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,One of the best phone!,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Checking for opinion combination


df_adj = pd.DataFrame(table1.Review_text)
df_vrb = pd.DataFrame(table1.Review_text)
for i in range(1,len(table1.columns)):
#     print(table1.columns[i]) 
    for j in range(0, len(table1.Review_text)):
        if table1.iloc[j,i] == 1:
            clean_rev = clean_review1(table1.loc[j,'Review_text'])
            for sent in clean_rev:
                adj = check_nn_adj(sent,table1.columns[i])
                vrb = check_nn_vrb(sent,table1.columns[i])
#                 print(adj,'===',vrb)
                sent = table1.loc[j,'Review_text'];noun=table1.columns[i]
#                 print('sent ====' ,sent)
#                 if len(adj) >=0:
#                     for _i in range(0,len(adj)):
#                         dist = _i[1] - 
#                 if len(adj)>0 and len(vrb) ==0:
#                     postag = nltk.pos_tag(nltk.word_tokenize(sent));
#                 print('adj == ',adj, postag)
                postag = nltk.pos_tag(nltk.word_tokenize(sent));
                res = nearest(postag,noun,adj,vrb)
#                 print(res)
                if res != []:
                    if res[2] == 'ADJ':
#                         print('enter')
                
                        if res[1]-1>=0 and postag[res[1]-1][1] in ['RB','RBR','RBS','RB']:
                            opinion = [postag[res[1]-1][0] +','+ res[0]]
                            dfadj.iloc[j,i]=opinion
#                             print('if1',opinion)

                        elif res[1]+1 < len(postag) and postag[res[1]+1][1] in ['JJ','JJS','RB','RBR','RBS']:
                                opinion = postag[res[1]+1][0] +','+ res[0]
                                dfadj.iloc[j,i]=opinion
#                                 print('elif1',opinion)

                        else:
                            dfadj.iloc[j,i]=res[0]
#                             print('else1',res[0])


                    elif res[2] == 'VRB':
                        
                        if res[1]-1>=0 and postag[res[1]-1][1] in ['VBD','VBG','VPN','RB']:
                            opinion = postag[res[1]-1][0] +','+ res[0]
                            dfvrb.iloc[j,i]=opinion
#                                 print('1',opinion,df_vrb.columns)
                            
                        elif len(postag)>res[1]+1 and postag[res[1]+1][1] in ['JJ','JJS','RB','RBR','RBS']:
                            opinion = postag[res[1]+1][0] +','+ res[0]
                            dfvrb.iloc[j,i]=opinion
    #                                 print('1',opinion,df_adj.columns)
                        else:
                            df_vrb.loc[j,i]=res[0]
    #                                 print('1',res[0],df_vrb.columns)


#                 print('next sentence..................')

In [25]:
dfadj.head(5)

,Review_text,phone,amazon,android,backup,battery,beast,bite,brand,budget,call,camera,charge,charger,dash,deal,delivery,design,device,display,experience,feature,flagship,game,headphone,heat,issue,lag,life,look,market,mobile,money,performance,phone.1,photo,picture,price,problem,processor,product,purchase,quality,ram,range,review,sale,screen,service,smartphone,software,sound,speaker,time,update,usage,user,value,video
0,Best phone ever,best,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,One of the best phone!,best,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,one plus 6 is extremely worth for it's price a...,best,amazon,0,0,0,0,0,0,0,0,0,0,0,0,"extremely,blowing",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,amazon,0,0,0,0,1,0,0,0,amazon,0,0,0,0,0,0,0,0,0,0,0,0
3,Good pictures,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Six months later: no complaints. OnePlus does ...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


'plus'

In [0]:
df1 = dfadj
df2 = dfvrb

In [0]:
#download dataframe from colab to local
# from google.colab import files
# df2.to_csv('./dfvrb.csv', sep = ',', index = False)
# files.download('dfvrb.csv')

In [0]:
# creating a new dataframe for accuracy
count=0
ind=0
column = ['word1','word2', 'freq']
newdf = pd.DataFrame(columns=column)
# newdf.columns = column

In [0]:
# calculating the freq of each opinion words from the previous step
for i in range(1,len(table1.columns)):
  count = 0; l = []
  for j in range(0, len(table1.Review_text)):
    
    if df1.iloc[j,i] != 0 and df1.iloc[j,i] != 1:
#         print(type(df1.iloc[j,i]),table1.columns[i] )
#         print('**enter**')
        if count == 0:
            words = df1.iloc[j,i].split(',')
#             print(words)
  #                 wrd = words.strip("'' ")
            if len(words) < 2:
              l.append(words[0].strip("'' "))
              l.append(''.strip("'' "))
            else:
              l.append(words[0].strip("'' "))
              l.append(words[1].strip("'' "))

        count = count+1
  l.append(count)
#   print(l)
  if len(l)==3:
    newdf = newdf.append(pd.Series(l, index=['word1','word2', 'freq']), ignore_index=True)
  ind= ind+1


In [0]:
#   df_adj.to_csv("df_adj_final.csv", encoding='utf-8', index=False)
#   df_vrb.to_csv("df_vrb_final.csv", encoding='utf-8', index=False)

In [0]:
# The below function check for thw opinion words by checking the previous and next words.
def next(postag, wrd1, wrd2):
    count=0
    for i, element in enumerate(postag):
#         print(postag)
#         print(wrd1, wrd2)
#         print(i)
        if i == 0 and len(postag)>1:
            if str.lower(wrd1) == str.lower(postag[i][0]):
                if str.lower(wrd2) == str.lower(postag[i+1][0]):
                    count = 1
                    break
            elif str.lower(wrd2) == str.lower(postag[i][0]):
                if str.lower(wrd1) == str.lower(postag[i+1][0]):
                    count = 1
                    break
        elif i == len(postag)-1:
            if str.lower(wrd1) == str.lower(postag[i][0]):
                if str.lower(wrd2) == str.lower(postag[i-1][0]):
                    count = 1
                    break 
                
            elif str.lower(wrd2) == str.lower(postag[i][0]):
                if str.lower(wrd1) == str.lower(postag[i-1][0]):
                    count = 1
                    break
                    
        elif i in range(1,len(postag)):
            if str.lower(wrd1) == str.lower(postag[i][0]):
                if str.lower(wrd2) == str.lower(postag[i-1][0]):
                    count = 1
                    break 
                
            elif str.lower(wrd2) == str.lower(postag[i][0]):
                if str.lower(wrd1) == str.lower(postag[i-1][0]):
                    count = 1
                    break
                    
            elif str.lower(wrd1) == str.lower(postag[i][0]):
                if str.lower(wrd2) == str.lower(postag[i+1][0]):
                    count = 1
                    break
                    
            elif str.lower(wrd2) == str.lower(postag[i][0]):
                if str.lower(wrd1) == str.lower(postag[i+1][0]):
                    count = 1
                    break
                    
        else:
            count = 0
            continue
    if count == 1:
      return 1
    else:
      return 0

In [0]:
# Calculating the total frequency of the opinion in the overall corpus
total=[];tt=0
for i in range(0,len(newdf.word1)):
    tt=0
    wrd1 = newdf.word1[i]
    wrd2 = newdf.word2[i]
    for j in range(0, len(table1.Review_text)):
        clean_rev = clean_review1(table1.loc[j,'Review_text'])
        for sent in clean_rev:
            postag = nltk.pos_tag(nltk.word_tokenize(sent));
            count = next(postag,wrd1,wrd2)
            tt = tt+count
            
    total.append(tt)

                
                

In [0]:
#backup

newdf1 = newdf 
newdf.shape

In [0]:
#adding the total count of the opinion words to the freq dataframe
newdf['total'] = total